In [352]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

In [353]:
df =pd.read_csv('spam.csv',encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [354]:
df = df[['v1', 'v2']]
# change the name of column
df.columns = ['label', 'text']
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# EDA

In [355]:
df.shape

(5572, 2)

In [356]:
df['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

In [357]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [358]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   5572 non-null   str  
 1   text    5572 non-null   str  
dtypes: str(2)
memory usage: 87.2 KB


# Convert the columns to number (label)

In [359]:
df['label'] = df['label'].map({
    'ham':0,
    'spam':1
})

In [360]:
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [361]:
print(df.columns)


Index(['label', 'text'], dtype='str')


#  Text Cleaning / Preprocessing

In [362]:
# import re (regula experession)
# used for search & replace inside text 
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', ' url ', text)# remove any start with http then replace with url (using sub)
    text = re.sub(r'\d+', ' number ', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['text'] = df['text'].apply(clean_text)


# Split the data into training and testing data

In [363]:
# split the data into training and testing
from sklearn.model_selection import train_test_split
x_raw = df['text'] # input featurs
y = df['label'] # target\ output

X_train, X_test, y_train, y_test = train_test_split( x_raw, y, test_size=0.2, random_state=42)

In [364]:
X_train.shape

(4457,)

In [365]:
X_test.shape


(1115,)

In [366]:
y_train.shape


(4457,)

In [367]:
y_test.shape

(1115,)

# TF-IDF vectorization for text feature extraction

In [ ]:
# Feature Engineering
from sklearn.feature_extraction.text import TfidfVectorizer

feature_extraction  = TfidfVectorizer(stop_words='english')  # ignore common words like 'the', 'a', 'is', etc.

#   covert the text to number

X_train_features = feature_extraction.fit_transform(X_train)# fit_transform is used to fit the model and transform the data
X_test_features = feature_extraction.transform(X_test) # this ensures that the model treats the test data as unknown data



# Evaluate the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# build the model
spam_model = MultinomialNB()

# train the model
spam_model.fit(X_train_features, y_train) # fit the model &   data training used for training the model

# 5. calculate the accuracy of the model
y_pred = spam_model.predict(X_test_features)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

Accuracy: 96.77%


In [370]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



# Exmine the model

In [ ]:
# example of a spam message
new_mail = ["Congratulations! You've won a $1000 Walmart gift card. Click here to claim your prize now!"]

# convert the new message to features using the same feature extraction method
new_mail_features = feature_extraction.transform(new_mail)

# make a prediction
prediction = spam_model.predict(new_mail_features)

print(f"The message is: {prediction[0]}")

The message is: 1
